<a href="https://colab.research.google.com/github/shaghayegh5ghasemi/supervised_fuzzy_clustering/blob/main/Supervised_Fuzzy_Clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn

In [2]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# load data
trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=1, shuffle=True, num_workers=0)

testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = DataLoader(testset, batch_size=1, shuffle=False, num_workers=0)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
# plot the image
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy() # convert from tensor
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

Files already downloaded and verified
Files already downloaded and verified


In [20]:
from IPython.core.display import Image

def make_window(img, label):
  windows = img.unfold(1, 3, 2).unfold(2, 3, 2) # unflod(dimension, size, step)
  selected_windows = []
  for n in range(2):
    i = torch.randint(0, 14, (1, 1))
    j = torch.randint(0, 14, (1, 1))
    temp = windows[:, i[0, 0], j[0, 0]].reshape((27, 1))
    selected_windows.append([temp, label])
  return selected_windows

def make_dataset():
  dataset = []
  for i, (inputs, labels) in enumerate(trainloader):
    if i == 1000:
      break
    dataset = dataset + make_window(inputs[0], labels[0])
  return dataset

In [21]:
dataset = make_dataset()

In [38]:
# initialize the centroids and the parameters
gamma = 1
sigma = 0.7
threshold = 0.0004
beta = 1

number_of_dimensions = 27
centroids = []
covariances = []
inverted_covariances = []

In [6]:
def calculate_mahanalobis_distance(icm, m, x):
  # D^2 = (x-m)^T * C^-1 * (x-m), mahanalobis distance formula
  s = x-m
  return float(s.T@icm@s)

In [39]:
def initialize_centroids(dataset, number_of_dimensions, centroids, inverted_covariances, covariances, gamma, sigma, threshold):
  number_of_centroids = len(centroids)
  for data in dataset:
    if number_of_centroids == 0:
      number_of_centroids += 1
      centroids.append(data)
      covariances.append(sigma*torch.eye(number_of_dimensions))
      inverted_covariances.append(torch.eye(number_of_dimensions)/sigma)
    else:
      distances = []
      for i in range(number_of_centroids):
        centroid = centroids[i]
        icm = inverted_covariances[i]
        distances.append(calculate_mahanalobis_distance(icm, centroid[0], data[0]))
      distances = -1*gamma*np.array(distances)
      RM = np.exp(distances)
      if max(RM) < threshold:
        number_of_centroids += 1
        centroids.append(data)
        covariances.append(sigma*torch.eye(number_of_dimensions))
        inverted_covariances.append(torch.eye(number_of_dimensions)/sigma)


initialize_centroids(dataset, number_of_dimensions, centroids, inverted_covariances, covariances, gamma, sigma, threshold)


In [42]:
def calculate_membership(icm, vi, x, gamma):
  distance = calculate_mahanalobis_distance(icm, vi, x)
  return np.exp(-1*gamma*distance)

In [49]:
def calculate_q_ij(centroids, inverted_covariances, classes, dataset, gamma):
  q = np.zeros((len(centroids), len(classes))) # i = number of clusters, j = number of classes
  for i in range(len(centroids)):
    centroid = centroids[i][0]
    icm = inverted_covariances[i]
    for j in range(len(classes)):
      numinator = 0
      denuminator = 0
      for data in dataset:
        vector = data[0]
        label = data[1]
        miu_ik = calculate_membership(icm, centroid, vector, gamma)
        if int(label) == j:
          numinator += miu_ik
        denuminator += miu_ik
      q[i][j] = numinator/denuminator
  return q

In [63]:
def update_centroids(centroids, inverted_covariances, classes, dataset, gamma, beta):
  q = calculate_q_ij(centroids, inverted_covariances, classes, dataset, gamma)
  for i in range(len(centroids)):
    centroid = centroids[i][0]
    icm = inverted_covariances[i]
    numinator = torch.zeros((27, 1))
    denuminator = 0
    for data in dataset:
      vector = data[0]
      label = data[1]
      miu_ik = calculate_membership(icm, centroid, vector, gamma)
      p_ik = q[i][int(label)]
      numinator += (miu_ik*p_ik)*vector
      denuminator += miu_ik*p_ik
    centroids[i][0] = (1-beta)*centroid + (beta/denuminator)*(numinator)

In [65]:
print(centroids[0])
update_centroids(centroids, inverted_covariances, classes, dataset, gamma, beta)
print("**************************")
print(centroids[0])

[tensor([[ 0.0183],
        [ 0.0122],
        [ 0.0113],
        [ 0.0113],
        [ 0.0060],
        [ 0.0099],
        [ 0.0123],
        [ 0.0041],
        [ 0.0050],
        [ 0.0251],
        [ 0.0182],
        [ 0.0155],
        [ 0.0199],
        [ 0.0135],
        [ 0.0155],
        [ 0.0206],
        [ 0.0117],
        [ 0.0123],
        [-0.0304],
        [-0.0391],
        [-0.0400],
        [-0.0341],
        [-0.0422],
        [-0.0404],
        [-0.0357],
        [-0.0430],
        [-0.0439]]), tensor(6)]
**************************
[tensor([[ 0.0308],
        [ 0.0242],
        [ 0.0236],
        [ 0.0232],
        [ 0.0179],
        [ 0.0221],
        [ 0.0243],
        [ 0.0150],
        [ 0.0157],
        [ 0.0288],
        [ 0.0208],
        [ 0.0179],
        [ 0.0222],
        [ 0.0153],
        [ 0.0175],
        [ 0.0228],
        [ 0.0125],
        [ 0.0130],
        [-0.0588],
        [-0.0689],
        [-0.0695],
        [-0.0637],
        [-0.0728],
        

In [ ]:
# for i in range(100): 
#     if labels[i] == 9: 
#       print(images[i])
#       #imshow(torchvision.utils.make_grid(images[i]))
#       print(' '.join('%5s' % classes[labels[i]]))

imshow(torchvision.utils.make_grid(images[0]))
lists = make_window(img, labels[55])
# print(labels[55])
# imshow(torchvision.utils.make_grid(lists[55][0]))
#print(lists[55][1])
# print(lists[55][0])
# print(len(lists[55][0]))

for i in range(len(lists)): 
  imshow(torchvision.utils.make_grid(lists[i][0]))
  print(' '.join('%5s' % classes[lists[i][1]]))


imshow(torchvision.utils.make_grid(dataset[120][0]))
print(' '.join('%5s' % classes[dataset[120][1]]))

# get some random training images
# dataiter = iter(trainloader)
# images, labels = dataiter.next()

# patches = make_window(images[0], labels[0])

# for i in range(len(patches)):
#   print(patches[i][0].shape)